# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [1]:
#Your code here
import numpy as np
import pandas as pd
from flatiron_stats import welch_t, welch_df, p_value_welch_ttest
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('multipleChoiceResponses_cleaned.csv')
df.head()

/Users/jrchen/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compile

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN,NaN,NaN
2,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,NaN,NaN
3,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,250000.0
4,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26394 entries, 0 to 26393
Columns: 230 entries, GenderSelect to AdjustedCompensation
dtypes: float64(15), object(215)
memory usage: 46.3+ MB


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [4]:
#H0: The salaries of Master's degrees is not greater than Bachelor's
#H1: The salaries of Master's degrees is greater than Bachelor's

In [5]:
#Your code here
df['FormalEducation'].head()

0    Bachelor's degree
1      Master's degree
2      Master's degree
3      Master's degree
4      Doctoral degree
Name: FormalEducation, dtype: object

In [6]:
df['FormalEducation'].unique()

array(["Bachelor's degree", "Master's degree", 'Doctoral degree', nan,
       "Some college/university study without earning a bachelor's degree",
       'I did not complete any formal education past high school',
       'Professional degree', 'I prefer not to answer'], dtype=object)

In [7]:
df['AdjustedCompensation'].head()

0         NaN
1         NaN
2         NaN
3    250000.0
4         NaN
Name: AdjustedCompensation, dtype: float64

In [8]:
df['AdjustedCompensation'].isna().sum()
# Have to handle these na values and i choose to drop them although it is a big size

22051

In [9]:
# Detect outliers
df['AdjustedCompensation'].describe()

count    4.343000e+03
mean     6.636071e+06
std      4.293996e+08
min     -7.351631e+01
25%      2.036942e+04
50%      5.381217e+04
75%      9.566608e+04
max      2.829740e+10
Name: AdjustedCompensation, dtype: float64

In [10]:
IQR = df['AdjustedCompensation'].quantile(0.75) - df['AdjustedCompensation'].quantile(0.25)
lower = df['AdjustedCompensation'].quantile(0.25) - (1.5*IQR)
upper = df['AdjustedCompensation'].quantile(0.75) + (1.5*IQR)

In [11]:
df_master = df.loc[df['FormalEducation']=="Master's degree", ['AdjustedCompensation']]
df_master.dropna(inplace=True)
df_bachelor = df.loc[df['FormalEducation']=="Bachelor's degree", ['AdjustedCompensation']]
df_bachelor.dropna(inplace=True)

In [12]:
#Take our outliers
df_master = df_master.loc[df_master['AdjustedCompensation']>lower]
df_master = df_master.loc[df_master['AdjustedCompensation']<upper]
df_bachelor = df_bachelor.loc[df_bachelor['AdjustedCompensation']>lower]
df_bachelor = df_bachelor.loc[df_bachelor['AdjustedCompensation']<upper]

In [13]:
p_value_welch_ttest(df_master['AdjustedCompensation'], df_bachelor['AdjustedCompensation'], two_sided=False)

4.757936189481171e-08

In [14]:
scs.ttest_ind(df_master['AdjustedCompensation'], df_bachelor['AdjustedCompensation'], equal_var=False)[1] / 2

4.757936186811995e-08

In [15]:
# The p-value is around 0 which less than 0.05. We can to reject the H0 hypothises
# Therefore, the salaries of Master's degrees is greater than Bachelor's

## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [16]:
#H0: The salaries of Doctoral's degrees is not greater than Bachelor's
#H1: The salaries of Doctoral's degrees is greater than Bachelor's
#Your code here
df_doctor = df.loc[df['FormalEducation']=="Doctoral degree", ['AdjustedCompensation']]
df_doctor = df_doctor.loc[df_doctor['AdjustedCompensation']>lower]
df_doctor = df_doctor.loc[df_doctor['AdjustedCompensation']<upper]
p_value_welch_ttest(df_doctor['AdjustedCompensation'], df_bachelor['AdjustedCompensation'], two_sided=False)

0.0

In [17]:
# The p-value is around 0 which less than 0.05. We can to reject the H0 hypothises
# Therefore, the salaries of Doctoral's degrees is greater than Bachelor's

## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [18]:
df = df.loc[df['AdjustedCompensation']>lower]
df = df.loc[df['AdjustedCompensation']<upper,['AdjustedCompensation', 'FormalEducation']]
df.dropna(inplace=True)
#df.head()

In [19]:
formula = 'AdjustedCompensation ~ C(FormalEducation)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                          sum_sq      df         F        PR(>F)
C(FormalEducation)  3.945484e+11     6.0  28.09295  4.337503e-33
Residual            9.882570e+12  4222.0       NaN           NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!